In [1]:
# Step 1: Create new clean environment
!conda create -n consumer-sentiment python=3.10 -y
!conda activate consumer-sentiment

# Step 2: Install everything
!pip install torch torchvision
!pip install transformers sentence-transformers
!pip install pandas==2.2.2
!pip install scikit-learn matplotlib seaborn wordcloud opencv-python
!pip install numpy==1.24.4
!pip install pyarrow


/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 kB 10.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 74.6 MB/s eta 0:00:00:00:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You 

In [2]:
import pandas as pd
import numpy as np
import torch
import torchvision


In [4]:
# ================================================================
# 4. Multimodal Sentiment Alignment
# ================================================================
# Purpose:
# - Compare BERT text sentiment with Facial Emotion sentiment
# - Identify mismatches or inconsistencies
# ================================================================

# STEP 1: Imports
import pandas as pd
import torch
import cv2
from torchvision import models, transforms

# STEP 2: Load Cleaned Review Sentiments
df = pd.read_csv('/home/mukhopadhyayd/Customer Sentiment/data/processed/reviews_with_sentiment.csv')  # must have 'bert_sentiment' column

# STEP 3: Load Facial Emotion CNN Model
# Assuming you already have 'best_model.pth' from your previous training
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 7)
model.load_state_dict(torch.load('/home/mukhopadhyayd/Customer Sentiment/data/processed/best_model.pth', map_location=torch.device('cpu')))
model.eval()

# Define facial emotion classes
emotion_classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# STEP 4: Helper Functions
def preprocess_image(image_path):
    """Preprocess FER-2013 image for prediction."""
    image = cv2.imread(image_path)
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((48, 48)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    return transform(image).unsqueeze(0)

def predict_emotion(image_tensor):
    """Predict the facial emotion from a preprocessed tensor."""
    with torch.no_grad():
        output = model(image_tensor)
        _, pred = torch.max(output, 1)
    return emotion_classes[pred.item()]

def align_sentiments(text_sentiment, face_emotion):
    """Check if text sentiment matches face emotion mapped sentiment."""
    mapping = {
        'Happy': 'POSITIVE',
        'Neutral': 'NEUTRAL',
        'Surprise': 'NEUTRAL',
        'Angry': 'NEGATIVE',
        'Disgust': 'NEGATIVE',
        'Sad': 'NEGATIVE',
        'Fear': 'NEGATIVE'
    }
    expected_sentiment = mapping.get(face_emotion, 'NEUTRAL')
    return text_sentiment == expected_sentiment

# STEP 5: Example: Compare a Random Text and Random Face
# -------------------------------------------------------

# Pick a sample review text and its sentiment
sample_text = df['full_review'].iloc[0]
sample_text_sentiment = df['bert_sentiment'].iloc[0]

# Pick a sample FER-2013 face image (replace with your path!)
image_path = '/home/mukhopadhyayd/Customer Sentiment/data/FER-2013/test/surprise/PrivateTest_10427382.jpg'  # <- adjust this path!
img_tensor = preprocess_image(image_path)
predicted_face_emotion = predict_emotion(img_tensor)

# Perform alignment check
alignment_result = align_sentiments(sample_text_sentiment, predicted_face_emotion)

# STEP 6: Output Results
# -------------------------------------------------------

print("Sample Review Text:", sample_text)
print("Predicted BERT Text Sentiment:", sample_text_sentiment)
print("Predicted Facial Emotion:", predicted_face_emotion)
print("Is Alignment Consistent?", alignment_result)


/home/mukhopadhyayd/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mukhopadhyayd/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Sample Review Text: Good Quality Dog Food I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
Predicted BERT Text Sentiment: POSITIVE
Predicted Facial Emotion: Angry
Is Alignment Consistent? False
